In [1]:
import numpy as np

class TicTacToe:
    def __init__(self):
        self.board = np.zeros((3, 3), dtype=int)
        self.current_player = 1

    def reset(self):
        self.board = np.zeros((3, 3), dtype=int)
        self.current_player = 1
        return self.get_state()

    def get_state(self):
        return self.board.flatten()

    def get_available_actions(self):
        return [i for i, val in enumerate(self.get_state()) if val == 0]

    def make_move(self, action):
        if self.get_state()[action] != 0:
            raise ValueError("Invalid move")
        row, col = action // 3, action % 3
        self.board[row, col] = self.current_player
        winner = self.check_winner()
        done = winner is not None
        reward = 0
        if done:
            if winner == 1: reward = 1
            elif winner == -1: reward = -1
            else: reward = 0
        self.current_player *= -1
        return self.get_state(), reward, done

    def check_winner(self):
        for i in range(3):
            if abs(self.board[i, :].sum()) == 3: return self.board[i, 0]
            if abs(self.board[:, i].sum()) == 3: return self.board[0, i]
        if abs(np.diag(self.board).sum()) == 3: return self.board[0, 0]
        if abs(np.diag(np.fliplr(self.board)).sum()) == 3: return self.board[0, 2]
        if not np.any(self.board == 0): return 0
        return None

In [2]:
import math

def minimax(board, player):
    game = TicTacToe()
    game.board = board
    winner = game.check_winner()
    if winner is not None:
        return winner * player
    best_score = -math.inf
    for action in game.get_available_actions():
        new_board = board.copy()
        row, col = action // 3, action % 3
        new_board[row, col] = player
        score = -minimax(new_board, -player)
        if score > best_score: best_score = score
    return best_score if best_score != -math.inf else 0

In [3]:
# --- 아래 함수를 코드에 추가하세요 ---

def test_minimax_evaluation():
    """
    특정 보드 상태에서 minimax 함수가 각 가능한 수의 가치를 어떻게 평가하는지 보여줍니다.
    """
    print("\n--- Minimax Evaluation Test ---")

    # 테스트할 보드 상태를 설정합니다. (X: 1, O: -1, 빈칸: 0)
    # 예시: X가 중앙(4)에, O가 좌상단(0)에 둔 상태
    #  O |   |
    # ---+---+---
    #    | X |
    # ---+---+---
    #    |   |
    test_board = np.array([
        [-1, -1, 0],
        [ 1, 1, -1],
        [ -1, -1, 1]
    ], dtype=int)

    # 현재 플레이어는 X (1)
    current_player = 1

    game = TicTacToe()
    game.board = test_board
    game.current_player = current_player

    print("Current Board State:")
    board_str = ""
    for i, cell in enumerate(game.get_state()):
        mark = 'X' if cell == 1 else 'O' if cell == -1 else str(i)
        board_str += f" {mark} "
        if (i+1) % 3 == 0:
            board_str += "\n" if i < 8 else ""
            if i < 8: board_str += "---+---+---\n"
    print(board_str)
    print(f"Current Player: 'X' ({current_player})")
    print("-" * 20)

    available_actions = game.get_available_actions()
    print(f"Available actions: {available_actions}\n")

    move_scores = {}

    # 모든 가능한 수에 대해 minimax 점수를 계산
    for action in available_actions:
        # 가상의 다음 보드 생성
        next_board = test_board.copy()
        row, col = action // 3, action % 3
        next_board[row, col] = current_player

        # 다음 수는 상대방(-1) 차례이므로, 상대방 입장에서의 점수를 계산
        # 그 점수에 -를 붙이면 현재 플레이어 입장에서의 점수가 됨
        score = -minimax(next_board, -current_player)
        move_scores[action] = score

    print("Minimax score for each possible move:")
    for move, score in move_scores.items():
        result = ""
        if score == 1:
            result = "-> Guaranteed Win"
        elif score == 0:
            result = "-> Leads to a Draw"
        elif score == -1:
            result = "-> Leads to a Loss"
        print(f" - Placing 'X' at position {move}: Score = {score} {result}")

    best_score = max(move_scores.values())
    optimal_moves = [move for move, score in move_scores.items() if score == best_score]

    print("\n" + "="*30)
    print(f"Conclusion:")
    print(f"The best possible outcome for 'X' from this state is a score of {best_score}.")
    print(f"Optimal move(s) for 'X' are: {optimal_moves}")
    print("="*30)


# --- 메인 실행 부분 수정 ---
if __name__ == "__main__":
    # Minimax 함수 테스트 실행
    test_minimax_evaluation()

    # Decision Transformer 모델 학습 및 실행 (원할 경우 주석 해제)
    # model = train()
    # play_game_with_model(model, CONTEXT_LENGTH)


--- Minimax Evaluation Test ---
Current Board State:
 O  O  2 
---+---+---
 X  X  O 
---+---+---
 O  O  X 
Current Player: 'X' (1)
--------------------
Available actions: [2]

Minimax score for each possible move:
 - Placing 'X' at position 2: Score = 0 -> Leads to a Draw

Conclusion:
The best possible outcome for 'X' from this state is a score of 0.
Optimal move(s) for 'X' are: [2]


In [4]:
# --- 아래 새로운 함수를 기존 코드에 추가하세요 ---

def analyze_move_reason(board, action, player):
    """
    주어진 보드(board)에서 플레이어(player)가 특정 수(action)를 두는 것의
    전략적 이유를 분석하여 문자열로 반환합니다.
    """
    # 1. 가상의 다음 보드 생성
    next_board = board.copy()
    row, col = action // 3, action % 3
    next_board[row, col] = player

    # --- 패턴 1: 즉시 승리하는 수인지 확인 ---
    game = TicTacToe()
    game.board = next_board
    if game.check_winner() == player:
        return "[승리] 이 수로 한 줄을 완성하여 즉시 승리합니다."

    # --- 패턴 2: 상대방의 승리를 막는 필수 방어 수인지 확인 ---
    # 내가 두려는 그 자리에 만약 상대방이 두었다면 상대가 이기는 상황이었는지 확인
    opponent_board = board.copy()
    opponent_board[row, col] = -player # 상대방이 두었다고 가정
    game.board = opponent_board
    if game.check_winner() == -player:
        return "[필수 방어] 상대방이 다음 수에 이기는 것을 막는 결정적인 수입니다."

    # --- 패턴 3 & 4: 공격적인 수(위협, 포크)인지 확인 ---
    # 내가 둔 이후의 보드(next_board)에서, 나의 돌 2개 + 빈칸 1개 라인이 몇 개인지 계산
    threat_lines = 0
    win_patterns = [
        # Rows
        [0, 1, 2], [3, 4, 5], [6, 7, 8],
        # Columns
        [0, 3, 6], [1, 4, 7], [2, 5, 8],
        # Diagonals
        [0, 4, 8], [2, 4, 6]
    ]
    flat_next_board = next_board.flatten()
    for pattern in win_patterns:
        line = [flat_next_board[i] for i in pattern]
        if line.count(player) == 2 and line.count(0) == 1:
            threat_lines += 1

    if threat_lines >= 2:
        return "[공격: 포크 생성] 두 개 이상의 공격 라인을 동시에 만들어 필승 전략을 구축합니다."
    if threat_lines == 1:
        return "[공격: 위협 생성] 다음 턴에 이길 수 있는 공격 라인(돌 2개)을 만듭니다."

    # --- 기타: 위의 특정 패턴에 해당하지 않는 경우 ---
    return "[전략적 수] 장기적으로 유리한 위치를 점하기 위한 수입니다 (Minimax 평가)."


# --- 기존의 test_minimax_evaluation 함수를 아래와 같이 수정하세요 ---

def test_minimax_evaluation():
    """
    특정 보드 상태에서 minimax 함수가 각 가능한 수의 가치를 어떻게 평가하고,
    그 이유가 무엇인지 함께 보여줍니다.
    """
    print("\n--- Minimax & Reason Analysis Test ---")

    # 테스트할 보드 상태 (X가 중앙, O가 좌상단)
    test_board = np.array([
        [-1, 0, 0],
        [ 0, 1, 0],
        [ 0, 0, 0]
    ], dtype=int)
    current_player = 1

    game = TicTacToe()
    game.board = test_board
    game.current_player = current_player

    # (이하 보드 출력 코드는 이전과 동일)
    print("Current Board State:")
    board_str = ""
    for i, cell in enumerate(game.get_state()):
        mark = 'X' if cell == 1 else 'O' if cell == -1 else str(i)
        board_str += f" {mark} "
        if (i+1) % 3 == 0:
            board_str += "\n" if i < 8 else ""
            if i < 8: board_str += "---+---+---\n"
    print(board_str)
    print(f"Current Player: 'X' ({current_player})")
    print("-" * 30)

    available_actions = game.get_available_actions()
    move_evaluations = {}

    for action in available_actions:
        score = -minimax(game.board, -current_player)
        # 각 수에 대한 이유 분석 추가
        reason = analyze_move_reason(game.board, action, current_player)
        move_evaluations[action] = {'score': score, 'reason': reason}

    print("Evaluation for each possible move:")
    # 점수가 높은 순으로 정렬하여 출력
    sorted_moves = sorted(move_evaluations.items(), key=lambda item: item[1]['score'], reverse=True)

    for move, eval_data in sorted_moves:
        score = eval_data['score']
        reason = eval_data['reason']
        print(f" - Move to {move}: Score = {score:2d} | Reason: {reason}")

    best_score = sorted_moves[0][1]['score']
    optimal_moves = [move for move, eval_data in sorted_moves if eval_data['score'] == best_score]

    print("\n" + "="*40)
    print(f"Conclusion:")
    print(f"Optimal move(s) for 'X' are: {optimal_moves} (Score: {best_score})")
    print(f"The primary reason for choosing these moves is their potential to create threats or forks.")
    print("="*40)

# 메인 실행 부분에 test_minimax_evaluation() 호출을 넣어두세요.
if __name__ == "__main__":
    test_minimax_evaluation()


--- Minimax & Reason Analysis Test ---
Current Board State:
 O  1  2 
---+---+---
 3  X  5 
---+---+---
 6  7  8 
Current Player: 'X' (1)
------------------------------
Evaluation for each possible move:
 - Move to 1: Score =  0 | Reason: [공격: 위협 생성] 다음 턴에 이길 수 있는 공격 라인(돌 2개)을 만듭니다.
 - Move to 2: Score =  0 | Reason: [공격: 위협 생성] 다음 턴에 이길 수 있는 공격 라인(돌 2개)을 만듭니다.
 - Move to 3: Score =  0 | Reason: [공격: 위협 생성] 다음 턴에 이길 수 있는 공격 라인(돌 2개)을 만듭니다.
 - Move to 5: Score =  0 | Reason: [공격: 위협 생성] 다음 턴에 이길 수 있는 공격 라인(돌 2개)을 만듭니다.
 - Move to 6: Score =  0 | Reason: [공격: 위협 생성] 다음 턴에 이길 수 있는 공격 라인(돌 2개)을 만듭니다.
 - Move to 7: Score =  0 | Reason: [공격: 위협 생성] 다음 턴에 이길 수 있는 공격 라인(돌 2개)을 만듭니다.
 - Move to 8: Score =  0 | Reason: [전략적 수] 장기적으로 유리한 위치를 점하기 위한 수입니다 (Minimax 평가).

Conclusion:
Optimal move(s) for 'X' are: [1, 2, 3, 5, 6, 7, 8] (Score: 0)
The primary reason for choosing these moves is their potential to create threats or forks.


In [5]:
# (이전의 TicTacToe, minimax 등의 함수와 클래스는 그대로 둡니다)

def print_board(board):
    # (이전과 동일)
    chars = {1: 'X', -1: 'O', 0: ' '}
    for i, row in enumerate(board):
        print(f" {chars[row[0]]} | {chars[row[1]]} | {chars[row[2]]} ")
        if i < 2:
            print("---+---+---")

def get_all_pre_win_states_no_canonical():
    """
    회전/대칭 중복 제거 없이, 모든 'X의 승리 직전' 보드를 생성합니다.
    """
    print("회전/대칭을 모두 포함하여 모든 'X의 승리 직전' 보드를 생성합니다...")
    q = [(np.zeros((3, 3), dtype=int), 1)]
    
    # ★★★ 변경점: Canonical Form 대신, 보드 자체를 기준으로 방문 여부 확인 ★★★
    visited_raw_states = set()
    all_game_states = []

    while q:
        board, player = q.pop(0)
        
        # 보드의 원본 상태를 튜플로 변환
        board_tuple = tuple(board.flatten())
        
        if board_tuple in visited_raw_states:
            continue
        visited_raw_states.add(board_tuple)
        all_game_states.append((board, player))
        
        game_instance = TicTacToe()
        game_instance.board = board
        if game_instance.check_winner() is None:
            for action in game_instance.get_available_actions():
                next_board = board.copy()
                row, col = action // 3, action % 3
                next_board[row, col] = player
                q.append((next_board, -player))
    
    # 필터링 로직은 이전과 동일
    pre_win_boards = []
    for board, player in all_game_states:
        if player != 1: continue
        game_instance.board = board
        if game_instance.check_winner() is not None: continue
        found_winning_move = False
        for action in game_instance.get_available_actions():
            temp_board = board.copy()
            row, col = action // 3, action % 3
            temp_board[row, col] = 1
            temp_game_instance = TicTacToe()
            temp_game_instance.board = temp_board
            if temp_game_instance.check_winner() == 1:
                found_winning_move = True
                break
        if found_winning_move:
            pre_win_boards.append(board)
    return pre_win_boards

def find_winning_moves(board, player):
    # (이전과 동일)
    winning_moves = []
    game = TicTacToe()
    game.board = board
    for action in game.get_available_actions():
        temp_board = board.copy()
        row, col = action // 3, action % 3
        temp_board[row, col] = player
        temp_game = TicTacToe()
        temp_game.board = temp_board
        if temp_game.check_winner() == player:
            winning_moves.append(action)
    return winning_moves

def analyze_winning_configurations_all_symmetries():
    """
    모든 대칭을 포함한 '승리 배치'들의 통계를 분석합니다.
    """
    # ★★★ 변경점: 새로운 데이터 생성 함수 호출 ★★★
    pre_win_boards = get_all_pre_win_states_no_canonical()
    print(f"총 {len(pre_win_boards)}개의 '승리 직전' 보드(대칭 포함)를 찾았습니다.")
    
    # 그룹화 및 분석 로직은 이전과 동일
    grouped_by_win_spot = {i: [] for i in range(9)}
    for board in pre_win_boards:
        winning_moves = find_winning_moves(board, 1)
        for move in winning_moves:
            grouped_by_win_spot[move].append(board)
            
    print("\n'승리 위치'별로 '승리 배치'들의 통계를 분석합니다...")
    
    for win_spot, boards in sorted(grouped_by_win_spot.items()):
        print("\n" + "="*50)
        print(f"★★★ 승리 위치(Winning Spot): {win_spot} ★★★")
        print("="*50)
        
        total_boards = len(boards)
        if total_boards == 0:
            print("이 위치를 승리점으로 하는 '승리 직전' 상태가 없습니다.")
            continue
            
        print(f"이 위치를 통해 승리할 수 있는 '승리 배치'의 수: {total_boards}개")
        
        position_counts = np.zeros((9, 3), dtype=int)
        for board in boards:
            flat_board = board.flatten()
            for pos in range(9):
                if flat_board[pos] == 0: position_counts[pos, 0] += 1
                elif flat_board[pos] == 1: position_counts[pos, 1] += 1
                elif flat_board[pos] == -1: position_counts[pos, 2] += 1
        
        print("\n'승리 위치'를 제외한 나머지 8개 칸의 돌 분포 빈도:")
        print("-" * 50)
        for pos in range(9):
            if pos == win_spot: continue
            counts = position_counts[pos]
            freq_empty = (counts[0] / total_boards) * 100
            freq_x = (counts[1] / total_boards) * 100
            freq_o = (counts[2] / total_boards) * 100
            print(f"  - 위치 {pos}: \t빈칸({freq_empty:5.1f}%) \tX({freq_x:5.1f}%) \tO({freq_o:5.1f}%)")


if __name__ == "__main__":
    analyze_winning_configurations_all_symmetries()

회전/대칭을 모두 포함하여 모든 'X의 승리 직전' 보드를 생성합니다...
총 1498개의 '승리 직전' 보드(대칭 포함)를 찾았습니다.

'승리 위치'별로 '승리 배치'들의 통계를 분석합니다...

★★★ 승리 위치(Winning Spot): 0 ★★★
이 위치를 통해 승리할 수 있는 '승리 배치'의 수: 232개

'승리 위치'를 제외한 나머지 8개 칸의 돌 분포 빈도:
--------------------------------------------------
  - 위치 1: 	빈칸( 25.0%) 	X( 42.7%) 	O( 32.3%)
  - 위치 2: 	빈칸( 25.0%) 	X( 41.8%) 	O( 33.2%)
  - 위치 3: 	빈칸( 25.0%) 	X( 42.7%) 	O( 32.3%)
  - 위치 4: 	빈칸( 24.1%) 	X( 44.4%) 	O( 31.5%)
  - 위치 5: 	빈칸( 37.1%) 	X( 15.1%) 	O( 47.8%)
  - 위치 6: 	빈칸( 25.0%) 	X( 41.8%) 	O( 33.2%)
  - 위치 7: 	빈칸( 37.1%) 	X( 15.1%) 	O( 47.8%)
  - 위치 8: 	빈칸( 24.1%) 	X( 45.3%) 	O( 30.6%)

★★★ 승리 위치(Winning Spot): 1 ★★★
이 위치를 통해 승리할 수 있는 '승리 배치'의 수: 147개

'승리 위치'를 제외한 나머지 8개 칸의 돌 분포 빈도:
--------------------------------------------------
  - 위치 0: 	빈칸( 19.0%) 	X( 57.1%) 	O( 23.8%)
  - 위치 2: 	빈칸( 19.0%) 	X( 57.1%) 	O( 23.8%)
  - 위치 3: 	빈칸( 38.1%) 	X( 15.0%) 	O( 46.9%)
  - 위치 4: 	빈칸( 19.0%) 	X( 56.5%) 	O( 24.5%)
  - 위치 5: 	빈칸( 38.1%) 	X( 15.0%) 	O( 46.9%)
  - 위치 6: 	빈칸( 

In [6]:
# (이전의 모든 함수와 클래스는 그대로 둡니다)

def analyze_pairwise_correlations():
    """
    각 승리 위치 그룹 내에서, 칸들 간의 상관 관계(조건부 빈도)를 분석합니다.
    """
    # 1. 모든 대칭을 포함한 '승리 직전' 보드 데이터 가져오기
    pre_win_boards = get_all_pre_win_states_no_canonical() # 이전 단계에서 만든 함수
    print(f"\n총 {len(pre_win_boards)}개의 '승리 직전' 보드를 기반으로 상관관계를 분석합니다.")
    
    # 2. '승리 위치'를 기준으로 보드들을 그룹화
    grouped_by_win_spot = {i: [] for i in range(9)}
    for board in pre_win_boards:
        winning_moves = find_winning_moves(board, 1)
        for move in winning_moves:
            grouped_by_win_spot[move].append(board)
            
    # 3. 각 그룹별로 상관관계 분석
    for win_spot, boards in sorted(grouped_by_win_spot.items()):
        print("\n" + "#"*60)
        print(f"###      승리 위치(Winning Spot): {win_spot} 에 대한 상관관계 분석      ###")
        print("#"*60)
        
        total_boards = len(boards)
        if total_boards == 0:
            continue
        
        other_positions = [p for p in range(9) if p != win_spot]

        # 4. 하나의 '기준점(anchor_pos)'을 잡고 분석 시작
        for anchor_pos in other_positions:
            print(f"\n--- 분석 기준점(Anchor): {anchor_pos} ---")
            
            # [기준점상태, 다른위치, 다른위치상태] 3차원 카운터
            # 상태 인덱스: 0:'X', 1:'O', 2:'빈칸'
            conditional_counts = np.zeros((3, 9, 3), dtype=int)
            anchor_state_totals = np.zeros(3, dtype=int) # 기준점의 상태별 총 개수

            for board in boards:
                flat_board = board.flatten()
                anchor_val = flat_board[anchor_pos]
                
                # 값(-1, 0, 1)을 인덱스(1, 2, 0)로 매핑
                anchor_idx = {1: 0, -1: 1, 0: 2}[anchor_val]
                anchor_state_totals[anchor_idx] += 1
                
                for other_pos in other_positions:
                    if other_pos == anchor_pos: continue
                    other_val = flat_board[other_pos]
                    other_idx = {1: 0, -1: 1, 0: 2}[other_val]
                    conditional_counts[anchor_idx, other_pos, other_idx] += 1
            
            # 5. 조건부 빈도 계산 및 출력
            for anchor_idx, anchor_char in enumerate(['X', 'O', '빈칸']):
                total = anchor_state_totals[anchor_idx]
                if total == 0: continue
                
                print(f"\n  [조건] '{anchor_char}'이(가) 위치 {anchor_pos}에 있을 때 ({int(total)}개 경우):")
                
                for other_pos in other_positions:
                    if other_pos == anchor_pos: continue
                    
                    counts = conditional_counts[anchor_idx, other_pos]
                    freq_x = (counts[0] / total) * 100
                    freq_o = (counts[1] / total) * 100
                    freq_empty = (counts[2] / total) * 100
                    
                    print(f"    - 위치 {other_pos}의 상태: \tX({freq_x:5.1f}%) \tO({freq_o:5.1f}%) \t빈칸({freq_empty:5.1f}%)")


if __name__ == "__main__":
    # 필요한 모든 함수들이 코드 상단에 정의되어 있다고 가정합니다.
    analyze_pairwise_correlations()

회전/대칭을 모두 포함하여 모든 'X의 승리 직전' 보드를 생성합니다...

총 1498개의 '승리 직전' 보드를 기반으로 상관관계를 분석합니다.

############################################################
###      승리 위치(Winning Spot): 0 에 대한 상관관계 분석      ###
############################################################

--- 분석 기준점(Anchor): 1 ---

  [조건] 'X'이(가) 위치 1에 있을 때 (99개 경우):
    - 위치 2의 상태: 	X( 75.8%) 	O( 16.2%) 	빈칸(  8.1%)
    - 위치 3의 상태: 	X( 24.2%) 	O( 43.4%) 	빈칸( 32.3%)
    - 위치 4의 상태: 	X( 23.2%) 	O( 45.5%) 	빈칸( 31.3%)
    - 위치 5의 상태: 	X( 13.1%) 	O( 50.5%) 	빈칸( 36.4%)
    - 위치 6의 상태: 	X( 23.2%) 	O( 44.4%) 	빈칸( 32.3%)
    - 위치 7의 상태: 	X( 11.1%) 	O( 53.5%) 	빈칸( 35.4%)
    - 위치 8의 상태: 	X( 25.3%) 	O( 42.4%) 	빈칸( 32.3%)

  [조건] 'O'이(가) 위치 1에 있을 때 (75개 경우):
    - 위치 2의 상태: 	X( 18.7%) 	O( 41.3%) 	빈칸( 40.0%)
    - 위치 3의 상태: 	X( 57.3%) 	O( 21.3%) 	빈칸( 21.3%)
    - 위치 4의 상태: 	X( 62.7%) 	O( 16.0%) 	빈칸( 21.3%)
    - 위치 5의 상태: 	X( 18.7%) 	O( 41.3%) 	빈칸( 40.0%)
    - 위치 6의 상태: 	X( 56.0%) 	O( 22.7%) 	빈칸( 21.3%)
    - 위치 7의 상태: 	X( 22.7%) 	O( 34.7%) 	빈